In [2]:

from simen_funksjoner import *
from read_csv import *

In [3]:
h2o.init(ip ='localhost', port = 54321)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment AdoptOpenJDK (build 11.0.10+9); OpenJDK 64-Bit Server VM AdoptOpenJDK (build 11.0.10+9, mixed mode)
  Starting server from /Users/simenvoldqvam/opt/anaconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/z4/blr9pbks7g1gwx7yzstbjlgw0000gn/T/tmpnjs7yqq7
  JVM stdout: /var/folders/z4/blr9pbks7g1gwx7yzstbjlgw0000gn/T/tmpnjs7yqq7/h2o_simenvoldqvam_started_from_python.out
  JVM stderr: /var/folders/z4/blr9pbks7g1gwx7yzstbjlgw0000gn/T/tmpnjs7yqq7/h2o_simenvoldqvam_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,"14 days, 2 hours and 42 minutes"
H2O_cluster_name:,H2O_from_python_simenvoldqvam_r4nopi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
#creating points from lat and lon
point = make_point(stores_train)
test_point = make_point(stores_test)

stores_gk_impuded_merged = stores_gk_impuded.merge(point,how='left', on='store_id')
stores_gk_impuded_merged_df = stores_gk_impuded_merged.drop(['lat','lon'], axis = 1)

stores_test_impuded_merged = stores_test_impuded.merge(test_point,how='left', on='store_id')
stores_test_impuded_merged_df = stores_test_impuded_merged.drop(['lat','lon'], axis = 1)

stores_gk_impuded_merged_df


,year,grunnkrets_id,revenue,area_km2
year,NaN,NaN,NaN,NaN
grunnkrets_id,NaN,1.000000,0.014588,0.067284
revenue,NaN,0.014588,1.000000,0.013161
area_km2,NaN,0.067284,0.013161,1.000000


In [11]:
closest = train_set_feat[['store_id','closeststore__distance_lv1_desc']]
closest_test = test_set_feat[['store_id','closeststore__distance_lv1_desc']]

df_train = stores_gk_impuded_merged_df.merge(closest,how='left',on='store_id')
df_test = stores_test_impuded_merged_df.merge(closest_test, how='left', on='store_id')

df_train.corr()





,year,grunnkrets_id,revenue,area_km2,closeststore__distance_lv1_desc
year,NaN,NaN,NaN,NaN,NaN
grunnkrets_id,NaN,1.000000,0.014588,0.067284,0.156894
revenue,NaN,0.014588,1.000000,0.013161,-0.051313
area_km2,NaN,0.067284,0.013161,1.000000,0.199384
closeststore__distance_lv1_desc,NaN,0.156894,-0.051313,0.199384,1.000000


In [6]:


stores_train_hf = h2o.H2OFrame(df_train)
stores_test_hf = h2o.H2OFrame(df_test)

stores_test_hf

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,chain_name,mall_name,grunnkrets_name,district_name,municipality_name,area_km2,point,closeststore__distance_lv1_desc
914206820-914239427-717245,2016,VÅLERENGA HALAL BURGER AS,1.1.1.0,Hamburger restaurants,3.0127e+06,STRØMSVEIEN 25 A,nan,nan,Vålerenga rode 4,Vålerenga,Oslo,0.0570274,POINT (10.787031199017804 59.9086723048657),0.000309021
916789157-916823770-824309,2016,BURGER KING MYREN,1.1.1.0,Hamburger restaurants,8.0614e+06,MYREN 1,BURGER KING,nan,Kjempa,Gulset,Skien,0.165993,POINT (9.588242910455039 59.2014671593755),0.000275592
913341082-977479363-2948,2016,BURGER KING STOVNER,1.1.1.0,Hamburger restaurants,3.01392e+06,STOVNER SENTER 3,BURGER KING,Stovner Senter,Stovner senter,Fossum,Oslo,0.236628,POINT (10.9245236479163 59.9621456195866),8.75107e-06
889682582-889697172-28720,2016,BURGER KING TUNGASLETTA,1.1.1.0,Hamburger restaurants,1.60121e+07,TUNGASLETTA 16,BURGER KING,nan,Strindheim 4,Strindheim,Trondheim,0.983436,POINT (10.461090999464 63.4207850720531),0.00512888
997991699-998006945-417222,2016,VULKAN BURGERBAR,1.1.1.0,Hamburger restaurants,3.01430e+06,AUD SCHØNEMANNS VEI 15,VULKAN BURGERBAR,nan,Lille Tøyen,Hasle,Oslo,0.449502,POINT (10.7851232590326 59.9211016682561),0.00398779
914931487-815162862-756427,2016,BURGER KING OSLO S,1.1.1.0,Hamburger restaurants,3.01310e+06,EKEBERGVEIEN 235,BURGER KING,nan,Bråten,Nordstrand,Oslo,0.313006,POINT (10.799906707586803 59.8610831417894),0.000753435
967062979-972338656-6209,2016,MCDONALD'S TRONDHEIM TORG,1.1.1.0,Hamburger restaurants,1.60112e+07,KONGENS GATE 9,MCDONALDS,Trondheim Torg,Midtbyen 11,Midtbyen,Trondheim,0.143946,POINT (10.393717924312 63.4303622432771),0.000420522
914631734-914748119-740036,2016,KVERNERIET & FRIENDS,1.1.1.0,Hamburger restaurants,3.01091e+06,KIRKEVEIEN 64 A,nan,nan,Majorstuen rode 9,Majorstuen,Oslo,0.191613,POINT (10.715469633727901 59.9292563375274),0.000573382
970976361-973961837-23171,2016,MCDONALD'S STRANDTORGET LILLEHAMMER,1.1.1.0,Hamburger restaurants,5.01061e+06,STRANDPROMENADEN 91,MCDONALDS,Strandtorget,Sorgendal,Sentrum vest,Lillehammer,0.39577,POINT (10.448162297388699 61.1151285614373),0.00113692
979425031-979584385-54031,2016,MCDONALD'S ÅLESUND STORSENTER,1.1.1.0,Hamburger restaurants,1.50403e+07,STORGATA 6,MCDONALDS,Ålesund Storsenter,Kipervika,Nørvøy ytre,Ålesund,0.120695,POINT (6.15981495473944 62.4722750992065),0.000971461


In [7]:

x = stores_train_hf.columns
y = 'revenue'
x.remove(y)

aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=y, training_frame= stores_train_hf) 

AutoML progress: |
18:38:23.573: _train param, Dropping bad and constant columns: [store_id, year, store_name]

███
18:38:42.554: _train param, Dropping bad and constant columns: [store_id, year, store_name]

████
18:39:14.602: GLM_1_AutoML_1_20221110_183823 [GLM def_1] failed: DistributedException from /127.0.0.1:54321: 'Java heap space', caused by java.lang.OutOfMemoryError: Java heap space
18:39:14.612: _train param, Dropping bad and constant columns: [store_id, year, store_name]


18:39:21.512: _train param, Dropping bad and constant columns: [store_id, year, store_name]

██
18:39:30.169: _train param, Dropping bad and constant columns: [store_id, year, store_name]

██
18:40:51.617: _train param, Dropping bad and constant columns: [store_id, year, store_name]


18:40:54.926: _train param, Dropping bad and constant columns: [store_id, year, store_name]

█
18:40:58.572: _train param, Dropping bad and constant columns: [store_id, year, store_name]

█
18:41:03.698: _train param, Droppi

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,4.7443957,0.1329015,4.633835,4.7382126,4.849847,4.9036736,4.59641
mean_residual_deviance,94.81467,16.930622,80.841385,82.8607,122.37299,98.87983,89.11842
mse,94.81467,16.930622,80.841385,82.8607,122.37299,98.87983,89.11842
null_deviance,608105.9,70453.0,557954.1,568141.75,726125.25,620840.3,567468.06
r2,0.6010359,0.0275771,0.6203435,0.6229901,0.5580615,0.5888737,0.6149105
residual_deviance,244101.14,46401.8,211804.42,214112.05,320861.97,255208.84,218518.38
rmse,9.708058,0.8428169,8.991183,9.102785,11.062233,9.943833,9.440255
rmsle,0.8039173,0.0175838,0.7841472,0.8097963,nan,0.8178083,nan


In [8]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
best_model = aml.get_best_model()

lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance,algo
StackedEnsemble_AllModels_1_AutoML_1_20221110_183823,9.7433,94.932,4.73752,nan,94.932,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_1_20221110_183823,9.75384,95.1374,4.75011,nan,95.1374,StackedEnsemble
GBM_1_AutoML_1_20221110_183823,10.0753,101.512,4.89076,nan,101.512,GBM
XGBoost_3_AutoML_1_20221110_183823,10.0882,101.772,5.01111,nan,101.772,XGBoost
DRF_1_AutoML_1_20221110_183823,10.1725,103.48,4.72126,0.780733,103.48,DRF
GBM_5_AutoML_1_20221110_183823,10.2865,105.813,4.91345,nan,105.813,GBM
XGBoost_1_AutoML_1_20221110_183823,10.3169,106.439,5.17026,nan,106.439,XGBoost
GBM_3_AutoML_1_20221110_183823,10.3178,106.457,4.93528,nan,106.457,GBM
GBM_2_AutoML_1_20221110_183823,10.3596,107.321,4.9574,0.809339,107.321,GBM
GBM_4_AutoML_1_20221110_183823,10.3674,107.483,4.95764,nan,107.483,GBM


In [9]:
m = h2o.get_model(lb[2,"model_id"])
print(m.varimp(use_pandas=True))


                           variable  relative_importance  scaled_importance  \
0                        chain_name         8.179964e+06           1.000000   
1                 municipality_name         1.999336e+06           0.244419   
2                sales_channel_name         9.712676e+05           0.118737   
3                         mall_name         3.883014e+05           0.047470   
4               plaace_hierarchy_id         3.142915e+05           0.038422   
5                     district_name         1.151176e+05           0.014073   
6                           address         1.034398e+05           0.012646   
7                   grunnkrets_name         7.637927e+04           0.009337   
8                             point         5.939657e+04           0.007261   
9   closeststore__distance_lv1_desc         3.570613e+04           0.004365   
10                         area_km2         1.894549e+04           0.002316   
11                    grunnkrets_id         9.512916

In [10]:
prediction = aml.leader.predict(stores_test_hf)
df = stores_test_hf.cbind(prediction)
df = df.as_data_frame(use_pandas=True)
result = df.loc[:,("store_id", 'predict')]
submission = result.rename(columns = {"store_id": "id", "predict": "predicted"})
submission.to_csv("h2o_with_closest_stores.csv", index = False)

submission

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/Users/simenvoldqvam/opt/anaconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'plaace_hierarchy_id' has levels not trained on: ["1.5.1.0"]
  warnings.warn(w)
/Users/simenvoldqvam/opt/anaconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'sales_channel_name' has levels not trained on: ["Bowling alleys"]
  warnings.warn(w)
/Users/simenvoldqvam/opt/anaconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'address' has levels not trained on: ["17. MAI GATA 6", "4. STRØM TERRASSE 13", "A B AARSTENS GATE 4", "AAGAARDS PLASS 2", "AAGAARDVEIEN 177", "AARSTADGATEN 6", "AASGAARDEN 4", "ABEL MEYERS GATE 13", "ABELSBORG GATE 10", "ADMIRAL BØRRESENS VEI 6 E", ...4323 not listed..., "ØYEVOLLVEIEN 6", "ØYGARDEN 18", "ØYGARDSBAKKEN 13 A", "ØYJORDSVEIEN 1", "ØYRAGATA 9", "ØYRAPLASSEN 9", "ØYRO 17", "ØYRO 2", "ØYRO 39", "ØYVIND LAMBES VEI 6"]
  warnings.warn(w)
/Users/simenvoldq

,id,predicted
0,914206820-914239427-717245,5.834358
1,916789157-916823770-824309,7.160918
2,913341082-977479363-2948,6.581743
3,889682582-889697172-28720,9.044083
4,997991699-998006945-417222,16.567129
...,...,...
8572,983164994-983173446-1571,2.753024
8573,917194602-917240809-839622,0.821965
8574,916236255-916275269-800265,1.341487
8575,998529794-975934594-45230,2.302510
